In [ ]:
import tensorflow as tf
import tensorflow_addons

In [ ]:
# from tensorflow.keras import layers
# augmentor = tf.keras.Sequential([
#     layers.experimental.preprocessing.RandomContrast(.2),
#     layers.experimental.preprocessing.RandomZoom(.2, .2),
#     layers.experimental.preprocessing.RandomRotation(.25),
#     layers.experimental.preprocessing.RandomTranslation(.2, .2)
# ])

In [ ]:
image_size = 224
@tf.function
def parse_function(filename):
    image_string = tf.io.read_file(filename)
    image = tf.io.decode_image(image_string, channels=3, expand_animations=False)
    image = tf.image.resize(image, (image_size, image_size))
    image = tf.cast(image, tf.float32) / 255.
    image = tf.image.adjust_saturation(image, 1.5)
    image = tf.clip_by_value(image, 0, 1)
    return image

In [ ]:
filenames = tf.io.gfile.glob('../input/plant-pathology-2021-fgvc8/test_images/*.jpg')
dataset = (tf.data.Dataset.from_tensor_slices(filenames)
                .map(parse_function).batch(4).prefetch(tf.data.AUTOTUNE))

In [ ]:
model = tf.keras.models.load_model(
    '../input/mobilenet040820/mobilenet_v3.h5')

In [ ]:
one_hot = {'healthy': [0, 0, 0, 0, 0, 1],
           'scab': [0, 1, 0, 0, 0, 0],
           'scab frog_eye_leaf_spot': [0, 1, 1, 0, 0, 0],
           'frog_eye_leaf_spot': [0, 0, 1, 0, 0, 0],
           'rust': [0, 0, 0, 1, 0, 0],
           'complex': [1, 0, 0, 0, 0, 0],
           'powdery_mildew': [0, 0, 0, 0, 1, 0],
           'rust frog_eye_leaf_spot': [0, 0, 1, 1, 0, 0],
           'frog_eye_leaf_spot complex': [1, 0, 1, 0, 0, 0],
           'scab frog_eye_leaf_spot complex': [1, 1, 1, 0, 0, 0],
           'powdery_mildew complex': [1, 0, 0, 0, 1, 0],
           'rust complex': [1, 0, 0, 1, 0, 0]}
num_classes = 6
decode_one_hot = {tuple(value): key for key, value in one_hot.items()}

In [ ]:
predictions = model.predict(dataset).round()
labels = []
for prediction in predictions:
    print(prediction)
    if prediction.sum() == 0.:
        labels.append('complex')
    else:
        for i in range(len(prediction), -2, -1):
            new_prediction = list(map(lambda x: int(x), prediction.copy()))
            if 0 <= i <= len(prediction)-1:   new_prediction[i] = 0     
            if 0 <= i+1 <= len(prediction)-1: new_prediction[i+1] = 0
            try: labels.append(decode_one_hot[tuple(new_prediction)])
            except: continue
            break
labels

In [ ]:
import pandas
submission = pandas.DataFrame({'image' : filenames, 'labels' : labels})
submission['image'] = submission['image'].apply(lambda filename: filename.split('/')[-1])
submission.to_csv('submission.csv', index=False)
submission.head()